# 03a - BigQuery Machine Learning (BQML) - Machine Learning with SQL

BigQuery has a number of machine learning algorithms callable directly from SQL.  This gives the convenience of using the common language of SQL to "CREATE MODEL …).  The library of available models is constantly growing and covers supervised, unsupervised, and time series methods as well as functions for evaluation - even anomaly detection from results, explainability and hyperparameter tuning.  A great starting point for seeing the scope of available methods is [user journey for models](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-e2e-journey).

In this demonstration, BigQuery ML (BQML) is used to create a logistic regression model.

**Prerequisites:**

-  01 - BigQuery - Table Data Source

**Overview:**

-  Train logistic regression model with BQML
   -  CREATE MODEL …. model_type="LOGISTIC_REG"
-  Review training information
   -  SELECT * FROM ML.TRAINING_INFO…
-  Evaluated the models performance
   -  SELECT * FROM ML.EVALUATE…
-  Review the classification errors with a confusion matrix
   -  SELECT * FROM ML.CONFUSION_MATRIX…
-  Create prediction for data in BigQuery
   -  SELECT * FROM ML.PREDICT

**Resources:**

-  [BigQuery ML (BQML) Overview](https://cloud.google.com/bigquery-ml/docs/introduction)
-  [Overview of BQML methods and workflows](https://cloud.google.com/bigquery-ml/docs/reference/standard-sql/bigqueryml-syntax-e2e-journey)
-  [BigQuery magics for jupyter notebooks](https://github.com/GoogleCloudPlatform/bigquery-notebooks/blob/main/notebooks/official/template_notebooks/bigquery_magic.ipynb)

**Related Training:**

-  todo

---
## Conceptual Architecture

<img src="architectures/statmike-mlops-02.png">

---
## Notes

This notebook uses [BigQuery Jupyter Magics](https://googleapis.dev/python/bigquery/latest/magics.html).  These allow parameters as inputs with `--params`, however, the dataset reference cannot be parameterized.  For this reason, the full qualifier of `project_id.dataset.table` is manually included and will need to be replaced if you change the variables `PROJECT_ID` and/or `DATANAME`.

---
## Setup

inputs:

In [1]:
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
DATANAME = 'digits'
NOTEBOOK = '03a'

---
## Train Model

Use BigQuery ML to train multiclass logistic regression model:

In [2]:
%%bigquery
CREATE OR REPLACE MODEL `digits.digits_lr`
OPTIONS
    (model_type='LOGISTIC_REG',
        auto_class_weights=TRUE,
        input_label_cols=['target'],
        data_split_col = 'custom_splits',
        data_split_method = 'CUSTOM'
    ) AS
SELECT * EXCEPT(splits, target_OE),
    CASE
        WHEN splits = 'TRAIN' THEN FALSE
        ELSE TRUE
    END AS custom_splits
FROM `digits.digits_prepped`

Query complete after 0.01s: 100%|██████████| 3/3 [00:00<00:00, 1877.77query/s]                        


""


Review the iterations from training:

In [3]:
%%bigquery
SELECT *
FROM ML.TRAINING_INFO(MODEL `digits.digits_lr`)
ORDER BY iteration

Downloading: 100%|██████████| 18/18 [00:01<00:00, 15.49rows/s]


,training_run,iteration,loss,eval_loss,learning_rate,duration_ms
0,0,0,0.194718,0.194983,0.2,8049
1,0,1,0.141744,0.142053,0.4,9506
2,0,2,0.087652,0.087618,0.8,7768
3,0,3,0.052650,0.052540,1.6,7871
4,0,4,0.033306,0.033338,3.2,10528
5,0,5,0.022404,0.022823,6.4,9085
6,0,6,0.016162,0.017603,12.8,8410
7,0,7,0.014686,0.016494,6.4,7751
8,0,8,0.013283,0.015009,6.4,7764
9,0,9,0.012306,0.014538,6.4,7976


---
## Evaluate Model

Review the model evaluation statistics on the Test/Train splits:

In [4]:
%%bigquery
SELECT 'TRAIN' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TRAIN'))
UNION ALL
SELECT 'TEST' as SPLIT, * FROM ML.EVALUATE (MODEL `digits.digits_lr`,
    (SELECT * FROM `digits.digits_prepped` WHERE SPLITS='TEST'))

Downloading: 100%|██████████| 2/2 [00:00<00:00,  2.01rows/s]


,SPLIT,precision,recall,accuracy,f1_score,log_loss,roc_auc
0,TRAIN,0.988314,0.988251,0.988292,0.988264,0.073722,0.999994
1,TEST,0.985833,0.982143,0.981818,0.983261,0.097423,1.000000


Review the confusion matrix for each split:

In [5]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TRAIN')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  8.97rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,152,0,0,0,0,0,0,0,0,0
1,1,0,148,0,0,0,0,0,0,1,1
2,2,0,0,142,0,0,0,0,0,0,0
3,3,0,0,0,137,0,1,0,0,1,0
4,4,0,0,0,0,155,0,0,0,1,0
5,5,0,0,0,0,0,137,1,0,0,1
6,6,0,0,0,0,0,0,141,0,0,0
7,7,0,0,0,0,0,0,0,138,0,1
8,8,0,4,0,0,0,1,0,0,134,0
9,9,0,1,0,1,0,1,0,0,1,151


In [6]:
%%bigquery
SELECT *
FROM ML.CONFUSION_MATRIX (MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits = 'TEST')
  );

Downloading: 100%|██████████| 10/10 [00:01<00:00,  7.65rows/s]


,expected_label,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9
0,0,14,0,0,0,0,0,0,0,0,0
1,1,0,14,0,0,0,0,0,0,0,0
2,2,0,0,20,0,0,0,0,0,0,0
3,3,0,0,0,19,0,1,0,1,0,0
4,4,0,0,0,0,11,0,0,1,0,0
5,5,0,0,0,0,0,23,0,0,0,0
6,6,0,0,0,0,0,0,16,0,0,0
7,7,0,0,0,0,0,0,0,18,0,0
8,8,0,0,0,0,0,0,0,0,14,0
9,9,0,0,0,0,0,0,0,0,0,13


---
## Predictions

Create a pandas dataframe with predictions for the test data in the table:

In [7]:
%%bigquery pred
SELECT *
FROM ML.PREDICT(MODEL `digits.digits_lr`,(
    SELECT *
    FROM `digits.digits_prepped`
    WHERE splits='TEST')
  )

Downloading: 100%|██████████| 165/165 [00:01<00:00, 154.63rows/s]


Review columns from the predictions - note that the query added columns with prefix `predicted_`

In [8]:
pred.columns

Index(['predicted_target', 'predicted_target_probs', 'p0', 'p1', 'p2', 'p3',
       'p4', 'p5', 'p6', 'p7', 'p8', 'p9', 'p10', 'p11', 'p12', 'p13', 'p14',
       'p15', 'p16', 'p17', 'p18', 'p19', 'p20', 'p21', 'p22', 'p23', 'p24',
       'p25', 'p26', 'p27', 'p28', 'p29', 'p30', 'p31', 'p32', 'p33', 'p34',
       'p35', 'p36', 'p37', 'p38', 'p39', 'p40', 'p41', 'p42', 'p43', 'p44',
       'p45', 'p46', 'p47', 'p48', 'p49', 'p50', 'p51', 'p52', 'p53', 'p54',
       'p55', 'p56', 'p57', 'p58', 'p59', 'p60', 'p61', 'p62', 'p63', 'target',
       'target_OE', 'splits'],
      dtype='object')

Print the first few rows for the columns related to the actual and predicted values:

In [9]:
pred[['target', 'predicted_target', 'predicted_target_probs', 'splits']].head()

,target,predicted_target,predicted_target_probs,splits
0,2,2,"[{'label': 2, 'prob': 0.49188777723089605}, {'...",TEST
1,8,8,"[{'label': 8, 'prob': 0.645210874009482}, {'la...",TEST
2,5,5,"[{'label': 5, 'prob': 0.9386099642013176}, {'l...",TEST
3,2,2,"[{'label': 2, 'prob': 0.9997278239183013}, {'l...",TEST
4,5,5,"[{'label': 5, 'prob': 0.9952850917917082}, {'l...",TEST


Notice the nested dictionary for predicted probabilities.  In BigQuery this is a Record type structure with nested fields for `label` and `prop`.  This is returned to the pandas dataframe as a nested dictionary.

The following code sorts the dictionary for the first record by `prop`:

In [10]:
[sorted(x, key = lambda x: x['label']) for x in [pred.predicted_target_probs[0]]]

[[{'label': 0, 'prob': 0.0006195211661560219},
  {'label': 1, 'prob': 0.007310554738387288},
  {'label': 2, 'prob': 0.49188777723089605},
  {'label': 3, 'prob': 0.03374825768502068},
  {'label': 4, 'prob': 1.3484418970530746e-05},
  {'label': 5, 'prob': 0.0011085826539046172},
  {'label': 6, 'prob': 0.0024442538806281494},
  {'label': 7, 'prob': 6.854491326185026e-05},
  {'label': 8, 'prob': 0.4594937619164089},
  {'label': 9, 'prob': 0.0033052613963661273}]]

---
## Remove Resources
see notebook "XX - Cleanup"